Managing Azure access can quickly become overwhelming: dozens of groups, hundreds of users, thousands of permissions. A simple, dynamic **tree visualization** instantly reveals structure, gaps, and potential risks.

If you can extract it — you can visualize it.

## 🧹 Group Membership View

>NOTE: below is test data just to show how visuals should look like

The first visualization shows **which users belong to which groups**:

In [1]:
@"
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "An example of Cartesian layouts for a node-link diagram of hierarchical data.",
  "width": 600,
  "height": 400,
  "padding": 5,
  "signals": [
    {"name": "labels", "value": true, "bind": {"input": "checkbox"}}
  ],
  "data": [
    {
      "name": "tree",
      "url": "permissionsData/userGroupData.json",
      "type": "json",
      "transform": [
        {"type": "stratify", "key": "id", "parentKey": "parent"},
        {
          "type": "tree",
          "method": "tidy",
          "size": [{"signal": "height"}, {"signal": "width - 100"}],
          "separation": "false",
          "as": ["y", "x", "depth", "children"]
        }
      ]
    },
    {
      "name": "links",
      "source": "tree",
      "transform": [
        {"type": "treelinks"},
        {
          "type": "linkpath",
          "orient": "horizontal",
          "shape": "diagonal"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "color",
      "type": "ordinal",
      "domain": [0, 1, 2],
      "range": ["#5e4fa2", "#3288bd", "#66c2a5"]
    }
  ],
  "marks": [
    {
      "type": "path",
      "from": {"data": "links"},
      "encode": {
        "update": {"path": {"field": "path"}, "stroke": {"value": "#ccc"}}
      }
    },
    {
      "type": "symbol",
      "from": {"data": "tree"},
      "encode": {
        "enter": {"size": {"value": 100}, "stroke": {"value": "#fff"}},
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "fill": {"scale": "color", "field": "depth"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "name"},
          "fontSize": {"value": 9},
          "baseline": {"value": "middle"}
        },
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "dx": {"signal": "datum.children ? -7 : 7"},
          "align": {"signal": "datum.children ? 'right' : 'left'"},
          "opacity": {"signal": "labels ? 1 : 0"}
        }
      }
    }
  ]
}
"@ | Out-Display -MimeType "application/vnd.vega.v5+json"

## 🛡️ Permissions View

The second visualization shows **who has what permissions** across resources:

In [2]:
@"
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "An example of Cartesian layouts for a node-link diagram of hierarchical data.",
  "width": 600,
  "height": 1200,
  "padding": 5,
  "signals": [
    {"name": "labels", "value": true, "bind": {"input": "checkbox"}}
  ],
  "data": [
    {
      "name": "tree",
      "url": "permissionsData/prodScopesAndPermissions.json",
      "type": "json",
      "transform": [
        {"type": "stratify", "key": "id", "parentKey": "parent"},
        {
          "type": "tree",
          "method": "tidy",
          "size": [{"signal": "height"}, {"signal": "width - 100"}],
          "separation": "false",
          "as": ["y", "x", "depth", "children"]
        }
      ]
    },
    {
      "name": "links",
      "source": "tree",
      "transform": [
        {"type": "treelinks"},
        {
          "type": "linkpath",
          "orient": "horizontal",
          "shape": "diagonal"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "color",
      "type": "ordinal",
      "domain": [0, 1, 2],
      "range": ["#5e4fa2", "#3288bd", "#66c2a5"]
    }
  ],
  "marks": [
    {
      "type": "path",
      "from": {"data": "links"},
      "encode": {
        "update": {"path": {"field": "path"}, "stroke": {"value": "#ccc"}}
      }
    },
    {
      "type": "symbol",
      "from": {"data": "tree"},
      "encode": {
        "enter": {"size": {"value": 100}, "stroke": {"value": "#fff"}},
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "fill": {"scale": "color", "field": "depth"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "name"},
          "fontSize": {"value": 9},
          "baseline": {"value": "middle"}
        },
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "dx": {"signal": "datum.children ? -7 : 7"},
          "align": {"signal": "datum.children ? 'right' : 'left'"},
          "opacity": {"signal": "labels ? 1 : 0"}
        }
      }
    }
  ]
}
"@ | Out-Display -MimeType "application/vnd.vega.v5+json"

## 🛠️ How the data is collected

The data is generated with two PowerShell scripts:

- one extracts **group memberships**
- another builds **role assignment relationships** between users, groups, and scopes.

In [ ]:
Install-Module Az
Install-Module ipmgmt

In [ ]:
$subscriptionDetails = @{}
$currentContext = Get-AzContext

$users = Get-AzADUser
$groups = Get-AzADGroup
$servicePrincipals = Get-AzADServicePrincipal
$applications = Get-AzADApplication

Get-AzSubscription | % {
    Select-AzSubscription -SubscriptionObject $_  | out-null
    $subscriptionDetails[$_.Name] = @{
        users = $users
        groups = $groups
        assignments = Get-AzRoleAssignment
        servicePrincipals = $servicePrincipals
        applications = $applications
    }
}

Set-AzContext -Context $currentContext | out-null

# process data for group membership visualization
$userIndex = @{}

$users | % {
    $userIndex[$_.UserPrincipalName] = $_
}

$rootName = 'groupsRoot'
$rootChildren = $groups | % {
    $g = $_
    $members = Get-AzADGroupMember -GroupObjectId $g.Id -WarningAction Ignore

    $children = @($members | % { [pscustomobject]@{name = $userIndex[$_.UserPrincipalName].DisplayName; value = 1 } })

    [pscustomobject]@{
        name = $g.DisplayName
        children = $children
        value = $children.Count
    }
}

[pscustomobject]@{
    name = $rootName
    children = $rootChildren
    value = 1
} | ConvertTo-Json -Depth 10 | Out-File 'permissionsData/userGroupData.json' -Force

In [ ]:
# data processing functions

function generateName($o) {
    if ($o -is [Microsoft.Azure.PowerShell.Cmdlets.Resources.MSGraph.Models.ApiV10.MicrosoftGraphUser]) {
        return "user`:$($o.UserPrincipalName)"
    }

    if ($o -is [Microsoft.Azure.PowerShell.Cmdlets.Resources.MSGraph.Models.ApiV10.MicrosoftGraphGroup]) {
        return "group`:$($o.DisplayName)"
    }

    if ($o -is [Microsoft.Azure.PowerShell.Cmdlets.Resources.MSGraph.Models.ApiV10.MicrosoftGraphServicePrincipal]) {
        return "srvPrincipal`:$($o.DisplayName)"
    }

    if ($o -is [Microsoft.Azure.PowerShell.Cmdlets.Resources.MSGraph.Models.ApiV10.MicrosoftGraphApplication]) {
        return "application`:$($o.DisplayName)"
    }
}

function Get-PermissionDetails {
    param(
        $InputObject, # $groups + $users + $servicePrincipals + $applications
        $SubscriptionName
    )

    $users = $InputObject[$SubscriptionName].users
    $groups = $InputObject[$SubscriptionName].groups
    $assignments = $InputObject[$SubscriptionName].assignments
    $servicePrincipals = $InputObject[$SubscriptionName].servicePrincipals
    $applications = $InputObject[$SubscriptionName].applications

    $rootChildren = @(($users + $groups + $servicePrincipals + $applications) | % {
            $id = $_.id
            $name = generateName $_
            $permissions = $assignments | ?  ObjectId -eq $id  | Group-Object -Property Scope
            $children = $permissions | % {
                $rdNames = $_.Group.RoleDefinitionName | % { @([pscustomobject]@{ name = $_; value = 1; children = @() }) }
                [pscustomobject]@{
                    name     = "scope`:$(($_.Name -split "/")[-1])"
                    children = @($rdNames)
                    value    = @($rdNames).Count
                } }
            $ret = [pscustomobject]@{
                name     = $name
                value    = @($children).Count
                children = @($children) #$children.Count -gt 1 ? $children : "[ $children ]"
            }
            $ret
        })

    [pscustomobject]@{
        name     = $SubscriptionName
        children = @($rootChildren) | ? { $_.value -gt 0 }
        value    = @($rootChildren).Count
    }
}


$subscriptionDetails = @{}
$currentContext = Get-AzContext

$users = Get-AzADUser
$groups = Get-AzADGroup
$servicePrincipals = Get-AzADServicePrincipal
$applications = Get-AzADApplication
$assignments = Get-AzRoleAssignment

Get-AzSubscription | % {
    Select-AzSubscription -SubscriptionObject $_  | out-null
    $subscriptionDetails[$_.Name] = @{
        users = $users
        groups = $groups
        assignments = Get-AzRoleAssignment
        servicePrincipals = $servicePrincipals
        applications = $applications
    }
}

Set-AzContext -Context $currentContext | out-null


# process data
Get-PermissionDetails -InputObject $subscriptionDetails -SubscriptionName 'some-subscription-name' | ConvertTo-Json -Depth 100 | Out-File "src/data/prodScopesAndPermissions.json"

However, the output data structures are not compatible with vega transforms, to adjust it, we have to run one more step, and then use the results as a source in the cells above, like this:

```json
      "url": "permissionsData/prodScopesAndPermissions_flat.json",
```

In [ ]:
$inputPath = "notebooks/en/permissionsData/userGroupData.json"
$outputPath = "notebooks/en/permissionsData/userGroupData_flat.json"

$tree = Get-Content $inputPath -Raw | ConvertFrom-Json

$flattened = @()
$idCounter = 0

function Flatten-Tree {
    param(
        [Parameter(Mandatory)] $Node,
        [Parameter()] $ParentId,
        [ref] $FlatList,
        [ref] $IdCounter
    )

    $IdCounter.Value++

    $currentId = $IdCounter.Value

    $FlatList.Value += [pscustomobject]@{
        id     = $currentId
        name   = $Node.name
        parent = $ParentId
    }

    foreach ($child in $Node.children) {
        Flatten-Tree -Node $child -ParentId $currentId -FlatList $FlatList -IdCounter $IdCounter
    }
}

Flatten-Tree -Node $tree -FlatList ([ref]$flattened) -IdCounter ([ref]$idCounter)

$flattened | ConvertTo-Json -Depth 5 | Out-File $outputPath -Force

Write-Output "Flattened data saved to $outputPath"